# Step 1: Install and Load Packages

In [ ]:
!pip install -r requirements.txt

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Set device (use CUDA if available for acceleration)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

# Step 2: Download NVIDIA Stock Data

In [ ]:
ticker = "NVDA"
stock = yf.Ticker(ticker)
hist = stock.history(period="2y")  # Past two years of data

# Display the first and last few rows of the dataset
print(hist.head())
print(hist.tail())

# Extract relevant market data (Open, High, Low, Close)
data = hist[['Open', 'High', 'Low', 'Close']].copy()

# Compute technical indicators
data['SMA_10'] = data['Close'].rolling(window=10).mean()  # Simple Moving Average
data['EMA_10'] = data['Close'].ewm(span=10, adjust=False).mean()  # Exponential Moving Average
data['ATR'] = (data['High'] - data['Low']).rolling(window=10).mean()  # Average True Range

# Fill missing values
data.bfill(inplace=True)

# Step 3: Data Normalization (Separately Normalize Prices and Indicators)

In [ ]:
scaler_price = MinMaxScaler(feature_range=(0, 1))
scaler_others = MinMaxScaler(feature_range=(0, 1))

price_columns = ['Open', 'High', 'Low', 'Close']
indicator_columns = ['SMA_10', 'EMA_10', 'ATR']

price_scaled = scaler_price.fit_transform(data[price_columns])
indicator_scaled = scaler_others.fit_transform(data[indicator_columns])

data_scaled = np.hstack((price_scaled, indicator_scaled))

# Create time series sequences
def create_sequences(dataset, look_back=10):
    X, Y = [], []
    for i in range(len(dataset) - look_back):
        X.append(dataset[i:(i + look_back), :])
        Y.append(dataset[i + look_back, -1])  # Predict next day's Close price
    return np.array(X), np.array(Y)

look_back = 10
X, Y = create_sequences(data_scaled, look_back)

# Split into training (80%) and testing (20%) sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
Y_train, Y_test = Y[:train_size], Y[train_size:]

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
Y_train = torch.tensor(Y_train, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
Y_test = torch.tensor(Y_test, dtype=torch.float32).to(device)

# Step 4: Define Transformer + LSTM Model

In [ ]:
class TransformerLSTMPredictor(nn.Module):
    def __init__(self, input_size=7, d_model=64, num_layers=2, num_heads=8, dim_feedforward=128):
        super(TransformerLSTMPredictor, self).__init__()

        # Transformer Encoder
        self.embedding = nn.Linear(input_size, d_model)
        self.norm = nn.LayerNorm(d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=num_heads, dim_feedforward=dim_feedforward, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # LSTM
        self.lstm = nn.LSTM(d_model, 32, num_layers=1, batch_first=True)

        # Fully Connected Layer
        self.fc = nn.Linear(32, 1)

    def forward(self, x):
        x = self.embedding(x)
        x = self.norm(x)
        x = self.transformer_encoder(x)
        x, _ = self.lstm(x)
        return self.fc(x[:, -1])  # Take the last time step's output

model = TransformerLSTMPredictor().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

# Step 5: Train Transformer + LSTM

In [ ]:
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    output = model(X_train)
    loss = criterion(output.squeeze(), Y_train)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 20 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.6f}")

# Step 6: Test and Evaluate Model

In [ ]:
model.eval()
with torch.no_grad():
    Y_pred = model(X_test).squeeze().cpu().numpy()
    Y_true = Y_test.cpu().numpy()

# Restore only the Close price
Y_pred_rescaled = scaler_price.inverse_transform(np.hstack([np.zeros((Y_pred.shape[0], 3)), Y_pred.reshape(-1, 1)]))[:, -1]
Y_true_rescaled = scaler_price.inverse_transform(np.hstack([np.zeros((Y_true.shape[0], 3)), Y_true.reshape(-1, 1)]))[:, -1]

# Compute RMSE and MAE
rmse = np.sqrt(mean_squared_error(Y_true_rescaled, Y_pred_rescaled))
mae = mean_absolute_error(Y_true_rescaled, Y_pred_rescaled)

print(f"Test RMSE: {rmse:.2f}")
print(f"Test MAE: {mae:.2f}")

# Step 7: Generate Future 5-Day Predictions

In [ ]:
future_predictions = []
input_seq = X_test[-1].cpu().numpy()  # Use the last test sample as the starting point

for _ in range(5):
    input_tensor = torch.tensor(input_seq, dtype=torch.float32).unsqueeze(0).to(device)
    with torch.no_grad():
        next_day = model(input_tensor).item()

    next_day_rescaled = scaler_price.inverse_transform(np.hstack([np.zeros((1, 3)), [[next_day]]]))[0, -1]
    future_predictions.append(next_day_rescaled)

    input_seq = np.roll(input_seq, -1, axis=0)
    input_seq[-1, -1] = next_day  # Update only Close price

print("Future 5-day predictions (Close prices):", future_predictions)

# Step 8: Save Predictions

In [ ]:
with open("41147011S.txt", "w") as f:
    for price in future_predictions:
        f.write(f"{price:.2f}\n")

print("Predictions saved to 41147011S.txt")